In [1]:
import pandas as pd
a = "https://firebasestorage.googleapis.com/v0/b/aarambh-aider.appspot.com/o/images%2FoutputFinal.csv?alt=media"
data = pd.read_csv(a, sep="\t")

In [2]:
import nltk 
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet') 


from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

from nltk.corpus import stopwords 
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 

VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'} 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def preprocess_sentences(text): 
  text = text.lower() 
  temp_sent =[] 
  words = nltk.word_tokenize(text) 
  tags = nltk.pos_tag(words) 
  for i, word in enumerate(words): 
      if tags[i][1] in VERB_CODES:  
          lemmatized = lemmatizer.lemmatize(word, 'v') 
      else: 
          lemmatized = lemmatizer.lemmatize(word) 
      if lemmatized not in stop_words and lemmatized.isalpha(): 
          temp_sent.append(lemmatized) 
          
  finalsent = ' '.join(temp_sent) 
  finalsent = finalsent.replace("n't", " not") 
  finalsent = finalsent.replace("'m", " am") 
  finalsent = finalsent.replace("'s", " is") 
  finalsent = finalsent.replace("'re", " are") 
  finalsent = finalsent.replace("'ll", " will") 
  finalsent = finalsent.replace("'ve", " have") 
  finalsent = finalsent.replace("'d", " would") 
  return finalsent 

In [4]:
data = data.set_index("Title")
data.head(10)

,Plot,Poster,Processed
Title,,,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",0,bartender work saloon serve drink customer fil...
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",0,moon paint smile face hang park night young co...
The Martyred Presidents,"The film, just over a minute long, is composed...",0,film minute long compose two shot first girl s...
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,0,last second consist two shot first shot set wo...
Jack and the Beanstalk,The earliest known adaptation of the classic f...,0,earliest known adaptation classic fairytale fi...
Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab...",0,alice follow large white rabbit find tiny door...
The Great Train Robbery,The film opens with two bandits breaking into ...,0,film open two bandit break railroad telegraph ...
The Suburbanite,The film is about a family who move to the sub...,0,film family move suburb hop quiet life thing s...
The Little Train Robbery,The opening scene shows the interior of the ro...,0,opening scene show interior robber den wall de...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer 
data["Processed"] = data["Processed"].fillna("")
# Vectorizing pre-processed movie plots using TF-IDF 
tfidfvec = TfidfVectorizer() 
tfidf_movieid = tfidfvec.fit_transform((data["Processed"]))

In [6]:
di = {}

In [16]:
tfidf_movieid.getrow(0).toarray().reshape(1, -1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [8]:
# !pip install progressbar
# from progressbar import ProgressBar
# pbar = ProgressBar()
from tqdm import tqdm_notebook as tqdm

In [14]:
# Finding cosine similarity between vectors 
from sklearn.metrics.pairwise import cosine_similarity



for i in tqdm(list(range(10000))):
  cos_sim = cosine_similarity(tfidf_movieid, tfidf_movieid.getrow(i).toarray().reshape(1, -1))
  di["index"+str(i)] = cos_sim.flatten().argsort()[::-1] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
import pickle
with open('data.p', 'wb') as fp:
  pickle.dump(di, fp, protocol=pickle.HIGHEST_PROTOCOL)